In [91]:
using Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")


   Resolving package versions...


  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Project.toml`
  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Project.toml`
  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Manifest.toml`

In [92]:
using JuMP
using HiGHS

In [123]:
import .KIRO2023


chemin_tiny = "instances/KIRO-tiny.json"
#chemin_small = "instances/KIRO-small.json"
#chemin_medium = "instances/KIRO-medium.json"
#chemin_large = "instances/KIRO-large.json"
#chemin_huge = "instances/KIRO-huge.json"

current_instance = KIRO2023.read_instance(chemin_tiny)

nb_WT = length(current_instance.wind_turbines) #Nombre de wind_turbine dans notre instance : card(V^s)=card(E0)
nb_SS = length(current_instance.substation_locations) #Nombre de substation dans notre instance Card(V^t)
CardVS=nb_SS
CardE0=CardVS
CardVT=nb_WT
CardS = length(current_instance.substation_types) #Card(S)
CardE0=nb_WT
CardQ0=length(current_instance.land_substation_cable_types)
CardES= CardVS*(CardVS-1)
CardQS=length(current_instance.substation_substation_cable_types)
CardET=CardVT*CardVS
OMEGA = length(current_instance.wind_scenarios)
#x : matrice taille VS x S, y0 : E0 x Q0, ys : ES x QS, z : VT x VS
#x[i, j] = ind(on construit type j sur emplacement i) (SS)
#y0[i,j] = ind(on construit type j sur route i) taille E0 x Q0 et ys[i,j,k] on construit type k entre  SS i et j (=ys[j,i,k]) taille VS x VS x QS
#z[i,j] = ind(turbine i reliée a SS j) taille WT * SS

#turb_links = zeros(Int,nb_WT) Les trois tableaux suivants sont ceux qu'on va mettre dans notre type "solution" pour créer le json -> Z
#st_cabl = zeros(Int,nb_SS) partie E_0 de y
#sub = Vector{KIRO2023.SubStation}() Les substations qu'on va construire-> x et partie E_s de y

function creesol(x,y0,ys,z)
 turb_links = zeros(Int,nb_WT)
    st_cable = zeros(Int,nb_SS)
    sub = Vector{KIRO2023.SubStation}()

    for i in 1:size(x, 1)  # Boucle sur les VS
        for j in 1:size(x, 2)  # Boucle pour les S
            if x[i,j]==1
                for k in 1:CardQ0
                    if y0[i,k] == 1
                        push!(sub, KIRO2023.SubStation(i,current_instance.substation_types[j],current_instance.land_cable_type[k]))
                    end

                end
            end
        end
    end
    for i in 1:size(ys,1)
        for j in 1:size(ys,2)
            for k in 1:size(ys,3)
                if ys[i,j,k]==1
                    st_cable[i, j] = k
                end
            end
        end
    end
    for i in 1:size(z,1)
        for j in 1:size(z,2)
            if z[i,j]==1
                turb_links[i]=j
            end
        end
    end
    return Solution(turb_links,st_cable,sub)
end



creesol (generic function with 1 method)

In [124]:
function variables_binaires!(model, tableau)
    for i in eachindex(tableau)
        @variable(model, tableau[i], Bin)
    end
end

function contrainteQ1!(model,mu,alpha,beta,M) # Quand on met la contrainte sur mu, alpha, beta existent deja, donc pas besoin de les def, @variable(model,mu)
    @constraint(model, mu >= 0)
    @constraint(model, mu <= M * alpha)
    @constraint(model, mu <= beta)
    @constraint(model, mu >= beta - (1 - alpha) * M)
end

function contrainteQ2!(model,mu,beta,M,y) #Attention, dire avant que y est binaire avec @variable(model, y, Bin) @variable(model,mu)
    @constraint(model, -beta <= M * y)
    @constraint(model, beta <= M * (1 - y))
    @constraint(model, mu >= 0)
    @constraint(model, mu >= beta)
    @constraint(model, mu <= M * (1 - y))
    @constraint(model, mu <= beta + M * y)   
end

function contrainteQ3!(model, mu, alpha, beta,M,x,y) # @variable(model, y, Bin)  @variable(model, x) @variable(model, mu)
    contrainteQ2!(model,x,beta-alpha,2*M,y)
    @constraint(model, mu == beta - x)
end


function contrainteQ4!(model,alpha,beta,gamma,A,B,y) #déclarer @variable(model,y) et pareil avec alpha, beta, gamma : les 4 doivent exister en tant que variable avant
    for i in 1:size(A, 1)
        @constraint(model, sum(A[i, j] * [x, y, z][j] for j in 1:3) <= B[i])
        @constraint(model, y >= alpha+gamma)
        @constraint(model, y >= beta+gamma)
    end
end

contrainteQ4! (generic function with 1 method)

In [125]:
"""M=10
alpha = 8
beta = 9
modeltest =  Model(HiGHS.Optimizer)
@variable(modeltest,y,Bin)
@variable(modeltest, x) 
@variable(modeltest,mu)
contrainteQ3!(modeltest,mu,alpha,beta,M,x,y)
@objective(modeltest,Min,mu)
optimize!(modeltest)"""

"M=10\nalpha = 8\nbeta = 9\nmodeltest =  Model(HiGHS.Optimizer)\n@variable(modeltest,y,Bin)\n@variable(modeltest, x) \n@variable(modeltest,mu)\ncontrainteQ3!(modeltest,mu,alpha,beta,M,x,y)\n@objective(modeltest,Min,mu)\noptimize!(modeltest)"

In [164]:
modelQ1 = Model(HiGHS.Optimizer)
@variable(modelQ1, x[1:CardVS, 1:CardS], Bin)
@variable(modelQ1, y0[1:CardE0, 1:CardQ0], Bin)
@variable(modelQ1, ys[1:CardVS, 1:CardVS, 1:CardQS], Bin)
@variable(modelQ1,z[1:CardVT ,1:CardVS], Bin)

3×2 Matrix{VariableRef}:
 z[1,1]  z[1,2]
 z[2,1]  z[2,2]
 z[3,1]  z[3,2]

In [165]:
#Je rajoute ttes les variables binaires (ou non) dont j'ai besoin
MQ = 50000
rx = [sum(current_instance.substation_types[i].rating * x[i, k] for k in 1:CardS) for i in 1:CardVS]
ryq0 = [sum(current_instance.land_substation_cable_types[i].rating * y0[i,k] for k in 1:CardQ0) for i in 1:CardVS]
somze = [ sum(z[k,i] for k in 1:CardVT) for i in 1:CardVS]
ryqs = [sum( j!=i ? sum( current_instance.substation_substation_cable_types[k].rating * ys[i,j,k]  for k in 1:CardQS) : 0 for j in 1:CardVS) for i in 1:CardVS] #Somme des rqyeq pour un v in Vs (yvvq=0)
ryqscf = [ i!=j ?  sum( current_instante.substation_substation_cable_types[k].rating * ys for k in 1:CardQS) : 0 for i in 1:CardVS, j in 1:CardVS]



UndefVarError: UndefVarError: `current_instante` not defined

In [166]:
#On construit tous les min de rsxvs/rqyeq : il y en a CardVS : pour chaque CardVS, il faut créer 3 variables. le mu, le x, et le yBin
@variable(modelQ1, MinCnx[1:CardVS])
@variable(modelQ1, MinCny[1:CardVS], Bin)
@variable(modelQ1, MinCn[1:CardVS])
for i in 1:CardVS
    contrainteQ3!(modelQ1,MinCn[i],-rx[i],-ryq0[i],MQ,MinCnx[i],MinCny[i])#model, mu, alpha, beta,M,x,y
end
#Attention, ici, MinCN est -min, il faudra donc mettre un plus dans la déf de Cn

In [167]:
#On crée les parties positives de Cn
@variable(modelQ1,PosCny[1:CardVS,1:OMEGA],Bin) #On a autant de variable binaire pour les partie positive que le nmbre de scenarios * VS
@variable(modelQ1,PosCn[1:CardVS, 1:OMEGA])
for i in 1:CardVS
    for j in 1:OMEGA #model,mu,beta,M,y
        contrainteQ2!(modelQ1,PosCn[i,j],current_instance.wind_scenarios[j].power_generation*somze[i]+MinCn[i],MQ,PosCny[i,j]) ##Le + est voulu
    end
end
@variable(modelQ1,Cn[1:OMEGA])
for j in 1:OMEGA
    @constraint(modelQ1,Cn[j] == sum( PosCn[i,j] for i in 1:CardVS))
end

In [168]:
#Cf sera un Cf[1:CardVS,1:OMEGA]
@variable(modelQ1, P1Cf[1:CardVS,1:OMEGA])
@variable(modelQ1, P1Cfy[1:CardVS,1:OMEGA])
for i in 1:CardVS
    for j in 1:OMEGA
        contrainteQ2!(modelQ1,P1Cf[i,j],current_instance.wind_scenarios[j].power_generation*somze[i]-ryqs[i],MQ,P1Cfy[i,j])#model,mu,beta,M,y
    end
end



In [169]:
#On crée les nouveaux tableaux dont on a besoin pour P2 de Cf :
